In [ ]:
!pip install -U datasets transformers trl peft bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134

## Gemma 2 Model Fine Tuning - Lora Model 학습

In [1]:
import torch
import pandas as pd
from datasets import Dataset, load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [2]:
mean_five_elements_hanja_prompt = pd.read_csv('./dataset/prompt/mean_five_elements_hanja_prompt.csv')
mean_five_elements_solving_prompt = pd.read_csv('./dataset/prompt/mean_five_elements_solving_prompt.csv')
korean_female_naming_prompt = pd.read_csv('./dataset/prompt/female_names_prompt_deduplication.csv')
korean_male_naming_prompt = pd.read_csv('./dataset/prompt/male_names_prompt_deduplication.csv')
korean_five_elements_naming_prompt = pd.read_csv('./dataset/prompt/mean_five_elements_name_prompt.csv')
name_meaning_prompt = pd.read_csv('./dataset/prompt/name_meaning_prompt.csv')

In [3]:
df = pd.concat([
    mean_five_elements_hanja_prompt,
    mean_five_elements_solving_prompt,
    korean_female_naming_prompt,
    korean_male_naming_prompt,
    korean_five_elements_naming_prompt,
    name_meaning_prompt
], ignore_index=True)

dataset = Dataset.from_pandas(df)

print(dataset)

Dataset({
    features: ['question', 'answer'],
    num_rows: 9314
})


In [4]:
def generate_prompt(dataset):
    prompt_list = []
    for i in range(len(dataset['question'])):
        prompt_list.append(r"""<bos><start_of_turn>user
{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(dataset['question'][i], dataset['answer'][i]))
    return prompt_list

In [5]:
base_model = "rtzr/ko-gemma-2-9b-it"
lora_model = "./lora-model"

In [6]:
lora_config = LoraConfig(
    lora_alpha = 8,
    lora_dropout = 0.1,
    r=16,
    target_modules=["k_proj", "o_proj", "v_proj", "q_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = 'right'

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    formatting_func=generate_prompt,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="./outputs",
        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=1,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/9314 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
100,1.009200
200,0.413400
300,0.366800
400,0.355800
500,0.332400
600,0.336900
700,0.329200
800,0.322000
900,0.310800
1000,0.311700


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

TrainOutput(global_step=3000, training_loss=0.30585841941833497, metrics={'train_runtime': 13000.621, 'train_samples_per_second': 0.923, 'train_steps_per_second': 0.231, 'total_flos': 8.911649571929395e+16, 'train_loss': 0.30585841941833497, 'epoch': 1.2883830792355595})

In [ ]:
trainer.model.save_pretrained(lora_model)

## Base Model과 Lora Model 병합

In [ ]:
base_model = "rtzr/ko-gemma-2-9b-it"
lora_model = "./lora-model"
fine_tuning_model = "./saju-naming"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model, device_map='auto', torch_dtype=torch.float16)

In [ ]:
model = PeftModel.from_pretrained(model, lora_model, device_map='auto', torch_dtype=torch.float16)

In [ ]:
model = model.merge_and_unload(progressbar=True)

## huggingface_hub 저장 또는 local 저장

In [ ]:
from huggingface_hub import interpreter_login

interpreter_login()

In [ ]:
# hugginface_hub 저장
model.push_to_hub('5KLetsGo/saju-naming')
tokenizer.push_to_hub('5KLetsGo/saju-naming')

In [ ]:
# fine-tuning model local 저장
model.save_pretrained(fine_tuning_model)